In [1]:
import numpy as np
import ray
import torch
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo import PPOTrainer
# from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.rllib.models.preprocessors import get_preprocessor
from ray.tune import register_env
from games.base import MeanFieldGame
from games.mfg_wrapper import MFGGymWrapper
from simulator.mean_fields.base import MeanField
from solver.base import Solver
from solver.policy.finite_policy import FiniteFeedbackPolicy

import time
import pickle
import pathlib

from solver.policy.random_policy import RandomFinitePolicy
from solver.stoc_reg_omd_graphon_solver import stoc_reg_DiscretizedGraphonExactOMDSolverFinite
from solver.stoc_reg_omd_graphon_solver_2 import stoc_reg_DiscretizedGraphonExactOMDSolverFinite_2
from solver.reg_omd_graphon_solver import reg_DiscretizedGraphonExactOMDSolverFinite
from solver.omd_graphon_solver import DiscretizedGraphonExactOMDSolverFinite
from solver.reg_graphon_solver import reg_DiscretizedGraphonExactSolverFinite
from solver.graphon_solver import DiscretizedGraphonExactSolverFinite
from evaluator.graphon_evaluator import DiscretizedGraphonEvaluatorFinite
from evaluator.reg_graphon_evaluator import reg_DiscretizedGraphonEvaluatorFinite
from evaluator.stochastic_evaluator import StochasticEvaluator
from games.finite.beach import BeachGraphon
from games.finite.cyber import CyberGraphon
from games.finite.cyber_het import HeterogeneousCyberGraphon
from games.finite.investment import InvestmentGraphon
from games.finite.sis import SISGraphon
from games.graphons import uniform_attachment_graphon, er_graphon, ranked_attachment_graphon, power_law_graphon, cutoff_power_law_graphon, sbm_graphon, exp_graphon
from simulator.graphon_simulator import DiscretizedGraphonExactSimulatorFinite
from simulator.stochastic_simulator import StochasticSimulator

ModuleNotFoundError: No module named 'ray.rllib.agents'

In [5]:
regularization = 1 
num_alphas = 30
game = BeachGraphon(**{"graphon": exp_graphon})
simulator = DiscretizedGraphonExactSimulatorFinite(**{})
evaluator = reg_DiscretizedGraphonEvaluatorFinite(**{'regularization':regularization,'num_alphas':num_alphas})

Now we can try to solve the game. 

In [6]:
def env_creator(env_config=None):
            return MFGGymWrapper(game, None, time_obs_augment=True)

register_env("MFG-v0", env_creator)

In [9]:
trainer = ppo.PPOTrainer(env="MFG-v0", config={
            'num_workers': 6,
            "gamma": 1,
            "entropy_coeff": 0.01,
            "clip_param": 0.2,
            "kl_target": 0.006,
        })



2024-04-19 20:20:18,377	WARNING ppo.py:141 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=6 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 666.


2024-04-19 20:20:25,202	WARNING trainer_template.py:185 -- `execution_plan` functions should accept `trainer`, `workers`, and `config` as args!
2024-04-19 20:20:25,205	WARNING util.py:57 -- Install gputil for GPU system monitoring.


In [11]:
trainer.train()

StopIteration: 

In [10]:
logs = []
verbose = True 
for iteration in range(20):
    log = trainer.train()
    if verbose:
        print("Loop {} mean {} ent {}".format(iteration, log['episode_reward_mean'],
                                                log['info']['learner']['default_policy']['entropy']))
    logs.append(log)
checkpoint = trainer.save()
trainer.load_checkpoint(checkpoint)

RayTaskError(AttributeError): [36mray::RolloutWorker.par_iter_next()[39m (pid=3983638, ip=10.1.102.239, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f1363deea30>)
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/util/iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 377, in gen_rollouts
    yield self.sample()
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 752, in sample
    batches = [self.input_reader.next()]
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 103, in next
    batches = [self.get_data()]
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 233, in get_data
    item = next(self._env_runner)
  File "/home/bcl/.conda/envs/py39/lib/python3.9/si

2024-04-19 20:21:32,667	ERROR worker.py:79 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::RolloutWorker.par_iter_next() (pid=3983639, ip=10.1.102.239, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fb86027fa30>)
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/util/iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 377, in gen_rollouts
    yield self.sample()
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 752, in sample
    batches = [self.input_reader.next()]
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 103, in next
    batches = [self.get_data()]
  File "/home/bcl/.conda/envs/py39/lib/python3.9/site-packages/ray/rllib/evaluation/sampler.py", line 233, in get_data
    item = next(